# Variational Quantum Algorithms in Q\#

## Abstract

In this sample, we will explore how the rich classical control provided by Q# can be used to efficiently write out variational quantum algorithms. In particular, we'll focus on the example of the _variational quantum eigensolver_, also known as VQE.

## Preamble

$
    \newcommand{\ket}[1]{\left|#1\right\rangle}
$

In [1]:
from itertools import product, repeat, starmap

In [2]:
import numpy as np
import qutip as qt
import qsharp

Preparing Q# environment...
.

For Q# code embedded in this notebook, it will be helpful to open a few namespaces before we proceed.

In [3]:
%%qsharp
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Math;

## Introducing the Variational Quantum Eigensolver

In variational quantum algorithms, rather than performing all of our computation on the quantum device itself, we use a quantum program to estimate some quantity, then use a classical optimizer to find inputs to our quantum program that minimize or maximize that quantity. That is, rather than thinking of classical computation purely as a pre-processing or post-processing step, our computation uses both classical and quantum computation together.

> **💡 TIP:** We could also consider using classical computation while qubits are still alive, rather than returning an estimate to a classical optimizer. This approach is indeed very useful, for instance in iterative phase estimation. For this sample, however, we'll focus on the variational case.


For example, consider the problem of finding the _ground state energy_ of a given operator $H$, often called a _Hamiltonian_. The ground state energy of $H$ is defined as its smallest eigenvalue $E_0$, as the Hamiltonian represents the possible energy configurations of a given physical system.

As stated, even though this is a minimization, we need to know the eigenvalues of $H$ to make any progress. It's pretty straightforward to rephrase the problem in terms of arbitrary states, however. In particular, the expectation value $\left\langle H \right\rangle H = \left\langle \psi | H | \psi \right\rangle$ must be at least as large as $E_0$. To see this, we can expand the expectation value in terms of the eigenvectors $\{\ket{\phi_i}\}$ of $H$, such that $H\ket{\phi_i} = E_i$.

Since the decomposition of $H$ into eigenvectors gives us a basis, we know that $\left\langle \phi_i | \phi_j \right\rangle$ is zero whenever $i \ne j$, allowing us to expand $\ket{\psi}$ into the eigenbasis of $H$ as $\ket{\psi} = \sum_i \alpha_i \ket{\phi_i}$ for some complex coefficients $\{\alpha_i\}$. Using this decomposition, we can expand the expectation $\left\langle \psi | H | \psi \right\rangle$ in terms of the eigenvalues of $H$:

$$
\begin{aligned}
    \left\langle \psi | H | \psi \right\rangle
        & = \sum_i |\alpha_i|^2 \left\langle \phi_i | H | \phi_i \right\rangle \\
        & = \sum_i |\alpha_i|^2 \left\langle \phi_i | E_i \phi_i \right\rangle \\
        & = \sum_i |\alpha_i|^2 E_i.
\end{aligned}
$$

Since each $|\alpha_i|^2$ is nonnegative, and since $\sum_i |\alpha_i|^2 = 1$, the above gives us that
$$
    E_0 = \min_i E_i \le \left\langle \psi | H | \psi \right\rangle \le \max_i E_i.
$$

Thus, we can rephrase the original problem as a minimization not just over eigenstates, but of all arbitrary states,
$$
    E_0 = \min_{\ket{\psi}} \left\langle \psi | H | \psi \right\rangle,
$$
where the minimum is achieved when $\ket{\psi} = \ket{\phi_0}$.

Using this rephrasing, the _variational quantum eigensolver_ algorithm is just the variational algorithm that we get by using a classical optimizer to find $\min_{\ket{\psi}} \left\langle \psi | H | \psi \right\rangle$. In pseudocode:

```
operation FindMinimumEigenvalue {
    Pick an initial guess |ψ⟩.
    until target accuracy reached {
        Estimate E = ⟨ψ | H | ψ⟩ using a quantum operation.
        Use a classical optimizer to pick the next |ψ⟩.
    }
    Return the minimum E that we found and the state |ψ⟩ that achieved that minimum.
}
```

**TODO: diagram**

To turn the above pseudocode into a real quantum program, we still need to figure out two things:

- How to estimate $\left\langle \psi | H | \psi \right\rangle$ for a given state $\ket{\psi}$.
- How to optimize over all quantum states $\ket{\psi}$.

In the rest of this sample, we'll see how to do each of these, and how you can use Q# to write a VQE implementation.

## Estimating expectation values of $H$

Before proceeding to see how to estimate $\left\langle \psi | H | \psi \right\rangle$, however, it helps to consider what that expectation value _is_. To do so, let's consider a concrete example of a two-qubit Hamiltonian, using QuTiP to construct a Python object representing $H$.

In [4]:
H = qt.Qobj([
    [-2, 0, 0, 3],
    [0, 7, 0, 1],
    [0, 0, -4, 0],
    [3, 1, 0, -1]
], dims=[[2, 2]] * 2)
H

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-2.  0.  0.  3.]
 [ 0.  7.  0.  1.]
 [ 0.  0. -4.  0.]
 [ 3.  1.  0. -1.]]

Here, `H` is a Python object representing the 4 × 4 matrix $H$. We can use the `eigenstates` method provided by QuTiP to quickly find the eigenvalues and eigenvectors of $H$:

In [5]:
H.eigenstates()

(array([-4.57776672, -4.        ,  1.43800535,  7.13976137]),
 array([Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[ 0.75726547]
         [ 0.05620122]
         [ 0.        ]
         [-0.65068458]]                                                    ,
        Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[ 0.]
         [ 0.]
         [-1.]
         [ 0.]]                                                            ,
        Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[-0.65152827]
         [ 0.13424187]
         [ 0.        ]
         [-0.74665256]]                                                    ,
        Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[0.04538633]
         [0.9893536 ]
         [0.        ]
         [0.13827341]]                                                    

In particular, we can use the `min` function provided by Python to minimize over the eigenvalues of $H$ and find its ground state $\ket{\phi_0}$.

In [6]:
min_energy, ground_state = min(zip(*H.eigenstates()), key=lambda eig: eig[0])
min_energy, ground_state

(-4.577766724471026,
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[ 0.75726547]
  [ 0.05620122]
  [ 0.        ]
  [-0.65068458]])

Here, `ground_state` represents the eigenvector $\ket{\phi_0}$ corresponding to the smallest eigenvalue $E_0$ of $H$. By the above argument, we would expect that $\left\langle \phi_0 | H | \phi_0 \right\rangle = E_0$. We can check that using QuTiP as well:

In [7]:
(ground_state.dag() * H * ground_state)[0, 0]

(-4.577766724471031+0j)

What's going on here? Effectively, for any operator $O$ such that $O^{\dagger} = O$, expressions like $\left\langle \psi | O | \psi \right\rangle$ represent another way of thinking about quantum measurement. In particular, if we think of the eigenvalues of $O$ as labels for its corresponding various eigenvectors, then $\left\langle \psi | O | \psi \right\rangle$ is the average label that we get when we measure $O$ in the basis of its eigenvectors.

> **💡 TIP:** This way of thinking about quantum measurement is sometimes called the _observable framework_, with $O$ being called an _observable_. We avoid using this terminology here to avoid confusion, however, as the expectation value of $O$ cannot be observed directly, but only inferred from repeated measurements.

For example, if $O = Z$, then the eigenstate $\ket{0}$ is labeled by its eigenvalue $+1$, while $\ket{1}$ is labeled by $-1$. The expectation value $\left\langle \psi | Z | \psi \right\rangle$ is then the probability of getting a `Zero` minus the probability of getting a `One`.

More generally, finding the expectation value of an arbitrary Pauli operator for an arbitrary input state is straightforward using the `Measure` and `EstimateFrequencyA` operations.

In [8]:
%%qsharp

operation EstimatePauliExpectation(pauli : Pauli[], preparation : (Qubit[] => Unit is Adj), nShots : Int) : Double {
    return 2.0 * EstimateFrequencyA(
        preparation,
        Measure(pauli, _),
        Length(pauli),
        nShots
    ) - 1.0;
}

Here, we've represented the state $\ket{\psi}$ by an operation `preparation` that prepares that state. Since each Pauli operator other than the identity operator has exactly two eigenvalues, $+1$ and $-1$, corresponding to `Zero` and `One` respectively, we can turn the estimate of the probability $p_0$ with which `Measure(pauli, _)` returns `Zero` into an expectation value $p_0 - p_1 = p_0 - (1 - p_0) = 2 p_0 - 1$.

For example, doing nothing prepares the state $\ket{0}$, so we get an expectation value of $1$:

In [9]:
%%qsharp

operation EstimateExpectationOfZero() : Double {
    return EstimatePauliExpectation(
        [PauliZ],
        NoOp,
        100
    );
}

In [10]:
EstimateExpectationOfZero.simulate()

1.0

Similarly, using `X` to prepare $\ket{1}$ gives us an expectation of $-1$, while using `H` to prepare $\ket{+} = \frac{1}{\sqrt{2}} (\ket{0} + \ket{1})$ gives an expectation value of $0$.

In [11]:
%%qsharp

operation EstimateExpectationOfOne() : Double {
    return EstimatePauliExpectation(
        [PauliZ],
        ApplyToEachCA(X, _),
        100
    );
}

In [12]:
EstimateExpectationOfOne.simulate()

-1.0

In [13]:
%%qsharp

operation EstimateExpectationOfPlus() : Double {
    return EstimatePauliExpectation(
        [PauliZ],
        ApplyToEachCA(H, _),
        100
    );
}

In [14]:
EstimateExpectationOfPlus.simulate()

0.020000000000000018

Note that in practice, we won't always get exactly 0 due to using a finite number of measurements.

In any case, to recap, it's easy to use a quantum program to find $\left\langle \psi | H | \psi \right\rangle$ in the special case that $H$ is a multi-qubit Pauli operator. What about the more general case? The linearity of quantum mechanics saves us again! It turns out that we can expand the expectation of any other operator in terms of expectations of Pauli operators. To see how that works, suppose that $H = 2 Z - X$.

In [15]:
2 * qt.sigmaz() - qt.sigmax()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 2. -1.]
 [-1. -2.]]

We can expand $\left\langle \psi | (2Z - X) | \psi \right\rangle$ by using linearity:

$$
    \left\langle \psi | (2Z - X) | \psi \right\rangle = 2 \left\langle \psi | Z | \psi \right\rangle - \left\langle \psi | X | \psi \right\rangle.
$$

Each of the terms in this expansion is something that we can estimate easily using our `EstimatePauliExpectation` operation from above.

In [16]:
%%qsharp

operation EstimateExpectation(terms : (Double, Pauli[])[], preparation : (Qubit[] => Unit is Adj), nShots : Int) : Double {
    mutable sum = 0.0;
    for (coefficient, pauli) in terms {
        set sum += coefficient * EstimatePauliExpectation(
            pauli, preparation, nShots
        );
    }
    return sum;
}

With this, we almost have everything we need to estimate expectations $\left\langle \psi | H | \psi \right\rangle$. We just need a way of finding the decomposition of $H$ into Pauli operators. Thankfully, QuTiP can help here as well.

In [17]:
def pauli_basis(n_qubits):
    scale = 2 ** n_qubits
    return {
        tuple(P): qt.tensor(*(p.as_qobj() for p in P)) / scale
        for P in product(qsharp.Pauli, repeat=n_qubits)
    }

In [18]:
def expand_in_pauli_basis(op):
    return [
        (coeff, list(label))
        for label, coeff in {
            label: (P.dag() * op).tr()
            for label, P in pauli_basis(n_qubits=len(op.dims[0])).items()
        }.items()
        if abs(coeff) >= 1e-10
    ]

For example, we can use the two functions above to find that $H = -3 𝟙 \otimes Z + 0.5 X \otimes 𝟙 + 1.5 X \otimes X - 0.5 X \otimes Z - 1.5 Y \otimes Y + 2.5 Z \otimes 𝟙 - 1.5 Z \otimes Z$:

In [19]:
H_decomposition = expand_in_pauli_basis(H)
H_decomposition

[(-3.0, [<Pauli.I: 0>, <Pauli.Z: 2>]),
 (0.5, [<Pauli.X: 1>, <Pauli.I: 0>]),
 (1.5, [<Pauli.X: 1>, <Pauli.X: 1>]),
 (-0.5, [<Pauli.X: 1>, <Pauli.Z: 2>]),
 (-1.5, [<Pauli.Y: 3>, <Pauli.Y: 3>]),
 (2.5, [<Pauli.Z: 2>, <Pauli.I: 0>]),
 (-1.5, [<Pauli.Z: 2>, <Pauli.Z: 2>])]

This decomposition is exactly what we need to pass as `terms` above. For example, to estimate the expectation of the $\ket{++}$ state, $\left\langle ++ | H | ++ \right\rangle$, we can pass `terms` to `EstimateExpectation`. In doing so, we'll use the name "energy" for our new operation, pointing to that expectation values of Hamiltonian operators $H$ represent the average energy of a system given the quantum state of that system.

In [20]:
%%qsharp

operation EstimateEnergyOfPlus(terms : (Double, Pauli[])[]) : Double {
    return EstimateExpectation(
        terms,
        ApplyToEachCA(H, _),
        100
    );
}

In [21]:
EstimateEnergyOfPlus.simulate(terms=H_decomposition)

2.080000000000001

This is pretty far from the minimum $E_0$ we found from the eigenvalue decomposition above, so in the next part we'll see one more trick we can use to write our VQE implementation.

## Preparing ansatz states

Recall that our goal is to use a classical optimizer to find the minimum energy $E_0$ of some Hamiltonian operator $H$:
$$
    E_0 = \min_{\ket{\psi}} \left\langle \psi | H | \psi \right\rangle.
$$

In practice, we can't reasonably optimize over all possible quantum states $\ket{\psi}$, so that the above optimization problem is intractable for all but the smallest systems. Instead, we note that we can find an upper-bound for $E_0$ by solving a simpler problem. Suppose that there is a set $A$ of states that are easier to prepare. Then,
$$
    E_0 = \min_{\ket{\psi}} \left\langle \psi | H | \psi \right\rangle \le \min_{\ket{\psi} \in A} \left\langle \psi | H | \psi \right\rangle.
$$

If we choose $A$ carefully so that $\ket{\phi_0} \in A$, this inequality will be tight. More often, however, $\ket{\phi_0}$ won't actually be in $A$, but will be close to some other state in $A$, giving us a reasonable approximation to $E_0$.

We say that $A$ is our _ansatz_ for running VQE; in this way, $A$ plays a similar role to a model in an ML training loop, representing what we believe to be a reasonable set of guesses for $\ket{\phi_0}$. One can get pretty involved with their choice of ansatz, but for this sample, we'll choose a really simple one, and set $A$ to be those states that can be prepared by a small number of Pauli rotations.

In [22]:
%%qsharp

operation PrepareAnsatz(axes : Pauli[][], angles : Double[], register : Qubit[]) : Unit is Adj {
    for (axis, angle) in Zipped(axes, angles) {
        Exp(axis, angle, register);
    }
}

Our minimization $\min_{\ket{\psi} \in A} \left\langle \psi | H | \psi \right\rangle$ is now a minimization over `angles` for some fixed list of Pauli rotation axes. Using `DumpMachine` and `DumpRegister`, we can visualize how this ansatz works for a few different choices of parameters `angles`, convincing ourselves that $A$ contains enough interesting states to find $E_0$.

In [23]:
%%qsharp

operation DumpAnsatz(axes : Pauli[][], angles : Double[]) : Unit {
    use register = Qubit[Length(angles)];
    within {
        PrepareAnsatz(axes, angles, register);
    } apply {
        DumpRegister((), register);
    }
}

In [24]:
ansatz_axes = [[qsharp.Pauli.Z, qsharp.Pauli.Z], [qsharp.Pauli.X, qsharp.Pauli.Y]]

In [25]:
DumpAnsatz.simulate(axes=ansatz_axes, angles=[1.2, 1.9])

|00⟩	-0.11714648149439554 + -0.3013185124022002𝑖
|11⟩	-0.3428991748354909 + -0.881988668778884𝑖

()

In [26]:
DumpAnsatz.simulate(axes=ansatz_axes, angles=[1.2, -0.7])

|00⟩	0.27714649751343473 + 0.7128628131458088𝑖
|11⟩	0.23343727454160576 + 0.600436064376938𝑖

()

At this point, it's helpful to write some new operations that directly estimate the energy of a state given a parameterization of our ansatz, rather than an opaque operation that prepares the state.

In [27]:
%%qsharp

operation EstimateEnergyAtAnsatz(terms : (Double, Pauli[])[], axes : Pauli[][], angles : Double[], nShots : Int) : Double {
    return EstimateExpectation(
        terms,
        PrepareAnsatz(axes, angles, _),
        nShots
    );
}

In [28]:
EstimateEnergyAtAnsatz.simulate(
    terms=H_decomposition,
    axes=ansatz_axes,
    angles=[1.2, 1.9],
    nShots=1000
)

0.7510000000000003

## Running VQE in Q\#

Using this new operation, we have something that looks much more like classical optimization problems that we may be used to! Indeed, we can simply use our favorite optimization algorithm to pick different values for `angles` until we find a good approximation for $E_0$. In this sample, we'll use the _SPSA algorithm_ to optimize `angles`, as this algorithm works especially well for objective functions that have some amount of noise, such as that added by taking a finite number of shots above.

We won't go through the details of SPSA here, but if you're interested to learn more, check out [`Optimization.qs`](../edit/Optimization.qs) in this folder to see how we implemented SPSA in Q#.

In [35]:
%%qsharp

open Microsoft.Quantum.Samples;

operation FindMinimumEnergy(terms : (Double, Pauli[])[], axes : Pauli[][], initialGuess : Double[], nShots : Int) : (Double[], Double) {
    let oracle = EstimateEnergyAtAnsatz(terms, axes, _, nShots);
    let options = DefaultSpsaOptions();
    return FindMinimumWithSpsa(
        oracle,
        initialGuess,
        options
    );
}

In [36]:
FindMinimumEnergy.simulate(terms=H_decomposition, axes=ansatz_axes, initialGuess=[1.2, 1.9], nShots=10_000)

Iteration 1:
	obj([1.3,2]) = 1.0886999999999998
	obj([1.0999999999999999,1.7999999999999998]) = 0.26560000000000006
Iteration 2:
	obj([-2.822261351356315,-2.3087386486436814]) = -4.4399
	obj([-3.008738648643681,-2.122261351356315]) = -3.9479
Iteration 3:
	obj([-1.2667200869599182,-3.864279913040078]) = 1.4102999999999994
	obj([-1.0877251490885658,-4.0432748509114305]) = 1.5343999999999998
Iteration 4:
	obj([-1.6220095588940617,-3.5089904411059347]) = 0.1336999999999997
	obj([-1.4481406468760574,-3.6828593531239386]) = 0.9106000000000005
Iteration 5:
	obj([-3.3896373535407216,-1.5713684773011478]) = -0.9934000000000003
	obj([-3.559631522698848,-1.7413626464592744]) = -2.0233
Iteration 6:
	obj([-5.8573036427035055,-3.8721423052520825]) = 1.42
	obj([-5.690411181491656,-4.039034766463932]) = 1.5356000000000005
Iteration 7:
	obj([-5.927245406092866,-3.8022005418627223]) = 1.2875999999999994
	obj([-6.091559604929135,-3.637886343026452]) = 0.7343999999999999
Iteration 8:
	obj([-6.971763352670

([-9.507013814290493, -2.424690641334752], -4.5384)

In [37]:
min_energy

-4.577766724471026